## See also _last_exam: LINQ + extension method is the principle

In [3]:

public static IEnumerable<char> Characters(this string input)
{
	foreach (var ch in input)
	{
		yield return ch;
	}
}

var test = "Hello!".Characters();
foreach(var c in test)
{
	Console.WriteLine(c);
}

// enumerator stuff ... see yield-return
IEnumerator<char> enumerator = test.GetEnumerator();

H
e
l
l
o
!


In [4]:
var test_filtered = from c in test where Char.IsLetter(c) select Char.ToUpper(c);

foreach(var c in test_filtered)
{
    Console.WriteLine(c);
}

H
E
L
L
O


### Generating (Mapping to) Objects

In [8]:
public class Person
{
    public char FavoriteLetter { get; set; }
    public string Name { get; set; }
    public int Age { get; set; }
}

var test_people = from c in test_filtered select new Person { FavoriteLetter = c, Name = "John", Age = 30 };

foreach(var p in test_people)
{
    Console.WriteLine(p.Name + " " + p.Age + " " + p.FavoriteLetter);
} 

John 30 H
John 30 E
John 30 L
John 30 L
John 30 O


Another Example: Rewrite as Extension Method

In [9]:
public class Person
{
    public int Id { get; set; }
    public string Name { get; set; }
    public int Age { get; set; }
}

public class Employee
{
    public int EmployeeId { get; set; }
    public string FullName { get; set; }
}


In [19]:
// Create a list of Person objects
List<Person> people = new List<Person>
{
    new Person { Id = 1, Name = "Alice", Age = 30 },
    new Person { Id = 2, Name = "Bob", Age = 40 },
    new Person { Id = 3, Name = "Charlie", Age = 25 },
    new Person { Id = 4, Name = "David", Age = 35 }
};

// Call the method to get a list of Employee objects
IEnumerable<Employee> employees = people.GetEmployees(30);

// Demo deferred execution
people.Add(new Person { Id = 5, Name = "Eve", Age = 40 });

// Print the Employee objects
foreach (var employee in employees)
{
    Console.WriteLine($"EmployeeId: {employee.EmployeeId}, FullName: {employee.FullName}");
}

public static IEnumerable<Employee> GetEmployees(this IEnumerable<Person> people, int minAge)
{
    // Use LINQ to filter the list and create new Employee objects
    //var filteredPeople = people.Where(p => p.Age >= minAge);
    //var filteredPeople = from p in people where p.Age >= minAge select p;

    var filteredEmployees = from p in people where p.Age >= minAge select new Employee{ EmployeeId = p.Id, FullName = p.Name };

    // Note: does not work (no construcutor with 2 arguments) - the above uses initializer
    //var filteredEmployees = from p in people where p.Age >= minAge select new Employee(p.Id, p.Name);

    foreach (var e in filteredEmployees)
    {
        yield return e;
    }

    /*foreach (var person in filteredPeople)
    {
        yield return new Employee
        {
            EmployeeId = person.Id,
            FullName = person.Name
        };
    }*/
}

EmployeeId: 1, FullName: Alice
EmployeeId: 2, FullName: Bob
EmployeeId: 4, FullName: David
EmployeeId: 5, FullName: Eve


An initializer in C# is a syntax that allows you to set the properties or fields of an object at the time of its creation, without explicitly calling a constructor for each property. This can be done using an object initializer or a collection initializer.

An object initializer allows you to create an instance of a class and initialize its properties in a single statement. This is particularly useful when you want to avoid writing multiple lines of code to set each property individually.

A collection initializer allows you to initialize a collection (such as a list, array, or dictionary) with a set of values at the time of its creation.

Both are used in the above.

## Student example from UE

In [24]:
record Student(string MatNr, string Name, string Subject, int[] Grades);

IEnumerable<Student> students = new List<Student>
{
    new Student("12345", "Alice", "SE", new int[] { 1, 2, 3 }),
    new Student("23456", "Bob", "Physics", new int[] { 4, 5, 6 }),
    new Student("34567", "Charlie", "Chemistry", new int[] { 7, 8, 9 })
};

In [28]:
var studentsFiltered = 
    from s in students 
    where s.Subject.Equals("SE", StringComparison.OrdinalIgnoreCase)
     select new {s.MatNr, s.Name};

foreach(var s in studentsFiltered)
{
    Console.WriteLine(s.MatNr + " " + s.Name);
}   

12345 Alice


>Important: the folllowing - using only extension methods

In [31]:
var studentsFilteredX =
    students.Where(s => s.Subject.Equals("SE", StringComparison.OrdinalIgnoreCase))
            .Select(s => new {s.MatNr, s.Name});

foreach(var s in studentsFilteredX)
{
    Console.WriteLine(s.MatNr + " " + s.Name);
} 

12345 Alice


In [34]:
var studentsBetterThanAverage =
    from s in students
    where s.Grades.All(g => g > 3)
    select s;

foreach(var s in studentsBetterThanAverage)
{
    Console.WriteLine(s.MatNr + " " + s.Name);
} 

23456 Bob
34567 Charlie


## let!

In [39]:
var studentsNamesAndAvgGrade =
    from s in students
    let avgGrade = s.Grades.Average()
    orderby avgGrade descending
    select new {s.Name, AvgGrade = avgGrade};

foreach(var s in studentsNamesAndAvgGrade)
{
    Console.WriteLine(s.AvgGrade + " " + s.Name);
} 

8 Charlie
5 Bob
2 Alice


In [46]:
var gradeStrings = new [] { "Sehr gut", "Gut", "Befriedigend", "Genügend", "Nicht genügend" };

var student = students.Single(s => s.MatNr == "12345");
var grades = from g in student.Grades select gradeStrings[g-1];

foreach(var g in grades)
{
    Console.WriteLine(g);
}

Sehr gut
Gut
Befriedigend
